In [2]:
import numpy as np
import pandas as pd
import glob, os, sys
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix

In [3]:
# the docword.*.txt files are formatted as:
# D
# W
# NNZ
# docID wordID count
os.chdir("CS674/HW4_topic_modeling/data")

In [4]:
glob.glob('*')

['docword.enron.txt',
 'docword.enron.txt_csc.npz',
 'docword.enron.txt_csr.npz',
 'docword.kos.txt',
 'docword.kos.txt_csc.npz',
 'docword.kos.txt_csr.npz',
 'docword.nips.txt',
 'docword.nips.txt_csc.npz',
 'docword.nips.txt_csr.npz',
 'docword.nytimes.txt',
 'docword.pubmed.txt_csc.npz',
 'docword.pubmed.txt_csr.npz',
 'docword.pubmed.zip',
 'mnist_all.mat',
 'processed',
 'raw',
 'readme.txt',
 'topic_output',
 'trunc_pubmed.txt',
 'vocab.enron.txt',
 'vocab.kos.txt',
 'vocab.nips.txt',
 'vocab.nytimes.txt',
 'vocab.pubmed.txt']

In [4]:
def store_files_to_disk(verbose: object = True) -> object:
    filenames = glob.glob("docword.*.txt")
    for fn in filenames:
        arr = np.loadtxt(fn, dtype=int,skiprows=3)
        coo = coo_matrix((arr[:,2], (arr[:,0], arr[:,1])))
        csc = csc_matrix(coo)
        csr = csr_matrix(coo)
        np.savez(fn+'_csc', data=csc.data, indices=csc.indices, indptr=csc.indptr, shape=csc.shape)
        np.savez(fn+'_csr', data=csr.data, indices=csr.indices, indptr=csr.indptr, shape=csr.shape)

In [5]:
def create_pubmed_coo(verbose=True):
    i = 0
    data, rows, cols = [], [], []
    # got from the readme file, can also be gotten from the first two lines of the file
    M = 300000+1
    N = 102660+1
    number_of_elements = 0
    coo = coo_matrix((M,N), dtype=int)
    file = "docword.nytimes.txt"
    # file = "trunc_pubmed.txt"
    with open(file, 'r') as csv:
        param_num = 0
        for row in csv:
            # print(i)
            if i<2:
                print(row)
            elif i==2:
                print(row)
                number_of_elements = int(row)
            elif i>2:
                # docID wordID count
                row_list = row.strip().split()
                rows.append(int(row_list[0]))
                cols.append(int(row_list[1]))
                data.append(int(row_list[2]))
                
                # keep memory use low
                if i % 10000000 == 0:
                    try:
                        print('i is : {}'.format(i))
                        print('lengths:\t\tcoo: {}\trows: {}\tcols: {}\tdata: {}'.format(coo.shape, len(rows), len(cols), len(data)))
                        print('Memory print 1:\tcoo: {}\trows: {}\tcols: {}\tdata: {}'.format(sys.getsizeof(coo), sys.getsizeof(rows), sys.getsizeof(cols), sys.getsizeof(data)))
                        coo = coo + coo_matrix((data, (rows, cols)), shape=(M,N), dtype=int)
                        data, rows, cols = [], [], []
                        print('Memory print 2:\tcoo: {}\trows: {}\tcols: {}\tdata: {}'.format(sys.getsizeof(coo), sys.getsizeof(rows), sys.getsizeof(cols), sys.getsizeof(data)))
                        print('\n------------------')
                    except ValueError:
                        print('raised an error!!!\nvalues:\tcoo: {}\trows: {}\tcols: {}\tdata: {}'.format(coo.shape, max(rows), max(cols), max(data)))
                        raise
            i += 1
            
        coo = coo + coo_matrix((data, (rows, cols)), shape=(M,N), dtype=int)
        print(coo)
        print(number_of_elements)
    return coo

In [11]:
# store_files_to_disk()
# coo = create_pubmed_coo()
# coo

In [9]:
fn = "docword.nytimes.txt"
csc = csc_matrix(coo)
np.savez(fn+'_csc', data=csc.data, indices=csc.indices, indptr=csc.indptr, shape=csc.shape)
csc

<300001x102661 sparse matrix of type '<class 'numpy.int32'>'
	with 69679427 stored elements in Compressed Sparse Column format>

In [10]:
# fn = "docword.pubmed.txt"
csr = csr_matrix(coo)
np.savez(fn+'_csr', data=csr.data, indices=csr.indices, indptr=csr.indptr, shape=csr.shape)
csr

<300001x102661 sparse matrix of type '<class 'numpy.int32'>'
	with 69679427 stored elements in Compressed Sparse Row format>

In [ ]:
fns = glob.glob('xa*')
# arr = np.loadtxt(fn, dtype=int,skiprows=3)
# coo = coo_matrix((arr[:,2], (arr[:,0], arr[:,1])))

In [ ]:
import pickle
with open(' pubmed_rows', 'wb'):
    pickle.dump(rows)
with open(' pubmed_cols', 'wb'):
    pickle.dump(rows)
with open(' pubmed_data', 'wb'):
    pickle.dump(rows)